# ETL

Importamos las librerías

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


Anteriormente hemos creado una copia de las hojas a utilizar del archivo de Excel homicidios, por ello tenemos separado hechos de víctimas. Ahora vamos a leer el archivo con Pandas

In [2]:
homicidios = pd.ExcelFile('homicidios.xlsx')


Creamos los Dataframes

In [3]:
hechos = homicidios.parse('HECHOS')
victimas = homicidios.parse('VICTIMAS')

## HECHOS

Leemos la información relevante

In [4]:
hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

Analizamos la información por columnas

In [5]:
hechos.head(5)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


Lo primero que consideraremos es la cantidad de información disponible en la columna altura. Decidimos borrarla pues solo tiene 129 filas con información y adicionalmente contamos con la dirección completa y la longitud y la latitud y la geo codificación plana XY (CABA)

In [6]:
hechos.drop(columns=['Altura'], inplace=True)

Nuestro segundo paso será depurar la geo codificación plana que nos permite ubicarnos en un plano bidimensional en un lugar preciso

In [7]:
hechos['XY (CABA)'] = hechos['XY (CABA)'].str.extract(r'\(([^)]+)')

In [8]:
hechos.head(5)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,98896.78238426 93532.43437792,-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,95832.05571093 95505.41641999,-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,NaN,ENTRE RIOS AV. 2034,1,106684.29090040 99706.57687843,-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,99840.65224780 94269.16534422,-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,106980.32827929 100752.16915795,-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


Ahora separaremos longitud y latitud con base en el espacio entre ambos

In [9]:
hechos[['geo plana X','geo plana Y']] = hechos['XY (CABA)'].str.split(expand=True)

Eliminamos la columna 'XY (CABA)'

In [10]:
hechos = hechos.drop(columns=['XY (CABA)'])

In [11]:
hechos.head(5)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Cruce,Dirección Normalizada,COMUNA,pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO,geo plana X,geo plana Y
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO,98896.78238426,93532.43437792
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS,95832.05571093,95505.41641999
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,NaN,ENTRE RIOS AV. 2034,1,-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO,106684.29090040,99706.57687843
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,-58.46503904,-34.68092974,MOTO-SD,MOTO,SD,99840.65224780,94269.16534422
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS,106980.32827929,100752.16915795


Como ya tenemos nuestras coordenadas geoespaciales planas y estas nos ayudaran a ubicarnos, vamos a eliminar las columnas calle y cruce, además nuestra dirección normalizada es el compuesto de las dos nombradas anteriormente + la altura de acuerdo a la disponibilidad de las 3

In [12]:
hechos = hechos.drop(columns=['Calle', 'Cruce'])

Eliminaremos la columna hora, pues no necesitamos los minutos en que ocurrió el accidente y HH nos muestra la franja horaria

In [13]:
hechos = hechos.drop(columns=['HORA'])

Como la columna HH está en formato objeto, vamos a cambiarla a tipo int, hay una fila que tiene problemas pues trae el valor 'SD', como nuestra franja horaria con más accidentes son las 7am valor a imputar esta fila con ese valor

In [14]:
hechos['HH'] = pd.to_numeric(hechos['HH'], errors='coerce')
hechos['HH'].fillna(7, inplace=True)
hechos['HH'].value_counts()
hechos['HH'] = hechos['HH'].astype(int)

In [15]:
hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HH                     696 non-null    int32         
 7   LUGAR_DEL_HECHO        696 non-null    object        
 8   TIPO_DE_CALLE          696 non-null    object        
 9   Dirección Normalizada  688 non-null    object        
 10  COMUNA                 696 non-null    int64         
 11  pos x                  696 non-null    object        
 12  pos y                  696 non-null    object        
 13  PARTI

Verificamos valores duplicados

In [16]:
duplicados =  hechos[hechos.duplicated(subset=['ID'])]
duplicados

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Dirección Normalizada,COMUNA,pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO,geo plana X,geo plana Y


## VICTIMAS

Leemos la información relevante

In [17]:
victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


Como podemos observar el tamaño de la base de datos hechos en términos de filas difiere de la base de datos víctimas, primero verificaremos duplicados

In [18]:
duplicados = victimas[victimas['ID_hecho'].duplicated()]
duplicados

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
30,2016-0041,2016-03-29,2016,3,29,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,SD,2016-03-30 00:00:00
99,2016-0126,2016-09-18,2016,9,18,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,60,SD
164,2017-0026,2017-02-26,2017,2,26,CONDUCTOR,AUTO,MASCULINO,19,2017-02-26 00:00:00
174,2017-0035,2017-03-23,2017,3,23,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,32,2017-03-23 00:00:00
175,2017-0035,2017-03-23,2017,3,23,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,30,2017-03-23 00:00:00
177,2017-0036,2017-03-29,2017,3,29,CONDUCTOR,MOTO,MASCULINO,20,2017-03-29 00:00:00
187,2017-0050,2017-04-28,2017,4,28,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,16,2017-04-28 00:00:00
238,2017-0108,2017-09-02,2017,9,2,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,10,2017-09-02 00:00:00
243,2017-0112,2017-09-10,2017,9,10,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,SD,2017-09-13 00:00:00
254,2017-0126,2017-10-14,2017,10,14,CONDUCTOR,AUTO,MASCULINO,36,2017-10-14 00:00:00


Debe haber alguna razón para ello, exploraremos algunas posiciones por ID_hecho

In [19]:
filtro = victimas[victimas['ID_hecho'] == '2016-0126']
print(filtro)

     ID_hecho      FECHA  AAAA  MM  DD                   ROL VICTIMA  \
98  2016-0126 2016-09-18  2016   9  18             CONDUCTOR    AUTO   
99  2016-0126 2016-09-18  2016   9  18  PASAJERO_ACOMPAÑANTE    AUTO   

         SEXO EDAD FECHA_FALLECIMIENTO  
98  MASCULINO   37                  SD  
99  MASCULINO   60                  SD  


In [20]:
filtro = victimas[victimas['ID_hecho'] == '2017-0035']
print(filtro)

      ID_hecho      FECHA  AAAA  MM  DD                   ROL VICTIMA  \
173  2017-0035 2017-03-23  2017   3  23             CONDUCTOR    AUTO   
174  2017-0035 2017-03-23  2017   3  23  PASAJERO_ACOMPAÑANTE    AUTO   
175  2017-0035 2017-03-23  2017   3  23  PASAJERO_ACOMPAÑANTE    AUTO   

          SEXO EDAD  FECHA_FALLECIMIENTO  
173  MASCULINO   28  2017-03-23 00:00:00  
174  MASCULINO   32  2017-03-23 00:00:00  
175  MASCULINO   30  2017-03-23 00:00:00  


Como podemos observar los duplicados no son precisamente duplicados, los ID están repetidos pues se refiere a casos donde varias personas fallecieron. Dado esto vamos a hacer un merge luego de la depuración con base en la tabla victimas para no perder información valiosa

Exploraremos ID_hecho

In [21]:
victimas['ID_hecho'].value_counts()

ID_hecho
2017-0035    3
2017-0050    2
2016-0041    2
2018-0015    2
2020-0063    2
            ..
2017-0115    1
2017-0116    1
2017-0118    1
2017-0120    1
2021-0096    1
Name: count, Length: 696, dtype: int64

Exploraremos FECHA

In [22]:
victimas['FECHA'].value_counts()

FECHA
2017-02-26    3
2016-02-28    3
2017-11-19    3
2017-12-11    3
2018-04-27    3
             ..
2017-11-23    1
2017-11-17    1
2017-11-16    1
2017-11-07    1
2021-12-15    1
Name: count, Length: 598, dtype: int64

Exploraremos AAAA

In [23]:
victimas['AAAA'].value_counts()

AAAA
2018    149
2016    146
2017    140
2019    104
2021     97
2020     81
Name: count, dtype: int64

Exploraremos MM

In [24]:
victimas['MM'].value_counts()

MM
12    81
11    68
8     67
1     62
5     60
2     59
6     58
3     56
4     52
10    52
7     51
9     51
Name: count, dtype: int64

Exploraremos DD

In [25]:
victimas['DD'].value_counts()

DD
20    32
17    30
3     28
11    28
9     27
27    27
14    27
10    26
25    26
23    26
12    26
28    26
15    25
29    25
18    24
2     24
4     23
22    23
7     23
19    23
26    22
6     20
21    20
24    19
5     19
16    18
13    18
1     18
30    17
8     14
31    13
Name: count, dtype: int64

Exploraremos ROL

In [26]:
victimas['ROL'].value_counts()

ROL
CONDUCTOR               330
PEATON                  267
PASAJERO_ACOMPAÑANTE     80
CICLISTA                 29
SD                       11
Name: count, dtype: int64

Vamos a imputar los valores SD como CONDUCTOR ya que es la mayor probabilidad y son únicamente 11 valores

In [27]:
victimas['ROL'].replace('SD', 'CONDUCTOR', inplace=True)

In [28]:
victimas['ROL'].value_counts()

ROL
CONDUCTOR               341
PEATON                  267
PASAJERO_ACOMPAÑANTE     80
CICLISTA                 29
Name: count, dtype: int64

Exploraremos VICTIMA

In [29]:
victimas['VICTIMA'].value_counts()

VICTIMA
MOTO         303
PEATON       267
AUTO          94
BICICLETA     29
SD             9
CARGAS         7
PASAJEROS      5
MOVIL          3
Name: count, dtype: int64

Dado que las victimas en mayor proporción son motos y que se asignó en ROL a las SD conductor, para ser congruentes asignaremos a los valores SD de victima el valor Moto

In [30]:
victimas['VICTIMA'].replace('SD', 'MOTO', inplace=True)

Exploraremos SEXO

In [31]:
victimas['SEXO'].value_counts()

SEXO
MASCULINO    545
FEMENINO     166
SD             6
Name: count, dtype: int64

Como en la tabla hechos no hay forma de identificar el SEXO, Vamos a proceder a asignar los valores SD con el sexo masculino que en proporciones es mucho mayor al femenino

In [32]:
for ind, elemento in enumerate(victimas['SEXO']):
    if elemento == 'SD':
        victimas['SEXO'][ind] = 'MASCULINO'
    else:
        pass

In [33]:
victimas['SEXO'].value_counts()

SEXO
MASCULINO    551
FEMENINO     166
Name: count, dtype: int64

Exploraremos EDAD

In [34]:
victimas['EDAD'].value_counts()

EDAD
SD    53
30    28
29    24
23    24
27    20
      ..
11     1
13     1
85     1
7      1
88     1
Name: count, Length: 86, dtype: int64

No hay forma de cruzar la información con la base de datos hechos. Una buena forma de calcular una edad estimada es hallar la media por sexo e imputar las filas faltantes con el resultado, para ello primero debemos cambiar el tipo de dato de la comuna EDAD verificando los componentes de la misma

Verificamos componentes

In [35]:
a = victimas['EDAD'].value_counts()
a

EDAD
SD    53
30    28
29    24
23    24
27    20
      ..
11     1
13     1
85     1
7      1
88     1
Name: count, Length: 86, dtype: int64

Procedremos a cambiar los 'SD' por 0

In [36]:
victimas['EDAD'].replace('SD', 0, inplace=True)
a = victimas['EDAD'].value_counts()
a

EDAD
0     53
30    28
29    24
23    24
27    20
      ..
11     1
13     1
85     1
7      1
88     1
Name: count, Length: 86, dtype: int64

Calculamos la media por edad excluyendo los 0 del calculo

In [37]:
filtro = victimas[victimas['EDAD'] != 0]
media = filtro.groupby('SEXO')['EDAD'].mean()
masculino = round(media.iloc[1],0)
femenino = round(media.iloc[0],0)
print(media)
print(femenino)
print(masculino)

SEXO
FEMENINO     50.761290
MASCULINO    39.550098
Name: EDAD, dtype: float64
51.0
40.0


Ahora imputaremos los valores faltantes en la columna EDAD con las medias por SEXO

In [38]:
mascara = (victimas['SEXO'] == 'FEMENINO') & (victimas['EDAD'] == 0)
victimas.loc[mascara, 'EDAD'] = femenino
filtro = victimas[victimas['EDAD'] == 0]

In [39]:
mascara2 = (victimas['SEXO'] == 'MASCULINO') & (victimas['EDAD'] == 0)
victimas.loc[mascara2, 'EDAD'] = masculino
filtro = victimas[victimas['EDAD'] == 0]

In [40]:
a = victimas['EDAD'].value_counts()
a

EDAD
40    53
30    28
23    24
29    24
28    20
      ..
11     1
13     1
85     1
7      1
88     1
Name: count, Length: 85, dtype: int64

Ahora verificaremos la fecha de fallecimiento

In [41]:
victimas['FECHA_FALLECIMIENTO'].value_counts()

FECHA_FALLECIMIENTO
SD                     68
2017-02-26 00:00:00     3
2017-01-16 00:00:00     3
2020-12-25 00:00:00     3
2019-12-18 00:00:00     3
                       ..
2018-01-19 00:00:00     1
2018-01-18 00:00:00     1
2018-01-14 00:00:00     1
2018-01-12 00:00:00     1
2022-01-02 00:00:00     1
Name: count, Length: 563, dtype: int64

Vamos a cambios los valores SD por la fecha 1999-12-31 00:00:00, para luego poder reemplazar el tipo de valor. También hay una fecha en otro formato 26/03/2019, la cambiaremos a 26/03/2019 00:00:00

In [42]:
victimas['FECHA_FALLECIMIENTO'] = victimas['FECHA_FALLECIMIENTO'].replace('SD', '1999-12-31 00:00:00' )
victimas['FECHA_FALLECIMIENTO'] = victimas['FECHA_FALLECIMIENTO'].replace('26/03/2019', '2019-03-26 00:00:00' )


In [43]:
a = victimas['FECHA_FALLECIMIENTO'].value_counts()
a

FECHA_FALLECIMIENTO
1999-12-31 00:00:00    68
2017-02-26 00:00:00     3
2017-01-16 00:00:00     3
2020-12-25 00:00:00     3
2019-12-18 00:00:00     3
                       ..
2018-01-19 00:00:00     1
2018-01-18 00:00:00     1
2018-01-14 00:00:00     1
2018-01-12 00:00:00     1
2022-01-02 00:00:00     1
Name: count, Length: 563, dtype: int64

Como es probable que la fecha de fallecimiento coincida con la fecha del accidente, asumiremos que fue le mismo día

In [44]:
for ind, elemento in enumerate(victimas['FECHA_FALLECIMIENTO']):
    if elemento == '1999-12-31 00:00:00':
        victimas['FECHA_FALLECIMIENTO'][ind] = victimas['FECHA'][ind]
    else:
        pass

In [45]:
a = victimas['FECHA_FALLECIMIENTO'].value_counts()
a

FECHA_FALLECIMIENTO
2016-10-25    4
2019-12-18    3
2018-04-27    3
2017-03-23    3
2018-08-03    3
             ..
2017-11-29    1
2017-11-28    1
2017-11-25    1
2017-11-23    1
2022-01-02    1
Name: count, Length: 607, dtype: int64

Ahora cambiamos el tipo de dato a fecha

In [46]:
import datetime

def to_date(text):
    date_string = str(text)
    date_format =  '%Y-%m-%d %H:%M:%S'
    date = datetime.datetime.strptime(date_string, date_format)
    return date

victimas['FECHA_FALLECIMIENTO'] = victimas['FECHA_FALLECIMIENTO'].apply(lambda x: to_date(x))


In [47]:
victimas['FECHA_FALLECIMIENTO'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 717 entries, 0 to 716
Series name: FECHA_FALLECIMIENTO
Non-Null Count  Dtype         
--------------  -----         
717 non-null    datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 5.7 KB


## MERGE 

Lo haremos con el dataframe victimas pues tiene más información que el dataset hechos y recordemos que según los análisis anteriores los duplicados se dan por la cantidad de fallecidos y su rol

In [48]:
merged = pd.merge(victimas, hechos, left_on= 'ID_hecho', right_on='ID')
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID_hecho               717 non-null    object        
 1   FECHA_x                717 non-null    datetime64[ns]
 2   AAAA_x                 717 non-null    int64         
 3   MM_x                   717 non-null    int64         
 4   DD_x                   717 non-null    int64         
 5   ROL                    717 non-null    object        
 6   VICTIMA_x              717 non-null    object        
 7   SEXO                   717 non-null    object        
 8   EDAD                   717 non-null    int64         
 9   FECHA_FALLECIMIENTO    717 non-null    datetime64[ns]
 10  ID                     717 non-null    object        
 11  N_VICTIMAS             717 non-null    int64         
 12  FECHA_y                717 non-null    datetime64[ns]
 13  AAAA_

Ahora vamos a chequear participantes que lo habíamos dejado pendiente para comparar con de la Victima del  dataset victimas

Verificamos las combinaciones donde se comience con SD en la columna participantes que equivale a la columna victima en el dataframe victimas

In [49]:
merged['PARTICIPANTES'].value_counts()

PARTICIPANTES
PEATON-PASAJEROS       105
MOTO-AUTO               84
MOTO-CARGAS             80
PEATON-AUTO             79
MOTO-PASAJEROS          50
MOTO-OBJETO FIJO        40
PEATON-CARGAS           38
AUTO-AUTO               33
PEATON-MOTO             30
AUTO-OBJETO FIJO        28
MOTO-MOTO               25
MULTIPLE                17
AUTO-CARGAS             15
AUTO-PASAJEROS          10
BICICLETA-CARGAS        10
MOTO-SD                  9
BICICLETA-PASAJEROS      8
BICICLETA-AUTO           8
SD-SD                    5
PEATON-BICICLETA         5
CARGAS-CARGAS            4
PEATON-SD                4
AUTO-SD                  4
PASAJEROS-PASAJEROS      3
SD-AUTO                  2
MOVIL-CARGAS             2
MOTO-BICICLETA           2
MOTO-OTRO                2
PEATON_MOTO-MOTO         2
MOTO-MOVIL               2
BICICLETA-OTRO           1
AUTO-MOVIL               1
BICICLETA-TREN           1
CARGAS-PASAJEROS         1
CARGAS-AUTO              1
PASAJEROS-AUTO           1
SD-MOTO       

Estas son las definiciones que comienzan con SD:
SD-SD
SD-AUTO
SD-MOTO
SD-CARGAS

Ahora vamos a realizar estos filtros para identificar cuando podemos reemplazar el valor SD con la definición de victima

In [50]:
filtro = merged[merged['PARTICIPANTES'] == 'SD-SD']
filtro[['PARTICIPANTES', 'VICTIMA_x']]

,PARTICIPANTES,VICTIMA_x
36,SD-SD,MOTO
77,SD-SD,MOTO
93,SD-SD,MOTO
141,SD-SD,MOTO
208,SD-SD,MOTO


In [51]:
filtro = merged[merged['PARTICIPANTES'] == 'SD-AUTO']
filtro[['PARTICIPANTES', 'VICTIMA_x']]

,PARTICIPANTES,VICTIMA_x
167,SD-AUTO,MOTO
221,SD-AUTO,MOTO


In [52]:
filtro = merged[merged['PARTICIPANTES'] == 'SD-MOTO']
filtro[['PARTICIPANTES', 'VICTIMA_x']]

,PARTICIPANTES,VICTIMA_x
280,SD-MOTO,MOTO


In [53]:
filtro = merged[merged['PARTICIPANTES'] == 'SD-CARGAS']
filtro[['PARTICIPANTES', 'VICTIMA_x']]

,PARTICIPANTES,VICTIMA_x
89,SD-CARGAS,MOTO


Como pudimos ver, no es posible imputar valores desde la columns victima_x del DataFrame victimas a la columna participantes.

Ahora vamos a seleccionar las columnas de tal forma que no se repitan, pues al hacer el merge tenemos información duplicada

In [54]:
hechos_victimas = merged[['ID_hecho', 'FECHA_x', 'AAAA_x','MM_x','DD_x','ROL','VICTIMA_x','SEXO','EDAD','FECHA_FALLECIMIENTO','N_VICTIMAS','HH','LUGAR_DEL_HECHO','TIPO_DE_CALLE','Dirección Normalizada','COMUNA','pos x','pos y','PARTICIPANTES','ACUSADO','geo plana X','geo plana Y']]


In [55]:
hechos_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID_hecho               717 non-null    object        
 1   FECHA_x                717 non-null    datetime64[ns]
 2   AAAA_x                 717 non-null    int64         
 3   MM_x                   717 non-null    int64         
 4   DD_x                   717 non-null    int64         
 5   ROL                    717 non-null    object        
 6   VICTIMA_x              717 non-null    object        
 7   SEXO                   717 non-null    object        
 8   EDAD                   717 non-null    int64         
 9   FECHA_FALLECIMIENTO    717 non-null    datetime64[ns]
 10  N_VICTIMAS             717 non-null    int64         
 11  HH                     717 non-null    int32         
 12  LUGAR_DEL_HECHO        717 non-null    object        
 13  TIPO_

Haremos un rename para facilidad en el manejo de la data y estética

In [56]:
rename = {'ID_hecho': 'ID', 'FECHA_x': 'Fecha', 'AAAA_x': 'Año', 'MM_x': 'Mes', 'DD_x': 'Dia', 'ROL': 'Rol', 'VICTIMA_x' : 'Victima', 'SEXO': 'Sexo', 'EDAD': 'Edad', 'FECHA_FALLECIMIENTO': 'Fecha_Fallecimiento', 'N_VICTIMAS': 'Numero_Victimas', 'HH': 'Franja_Horaria', 'LUGAR_DEL_HECHO': 'Lugar_Del_Hecho', 'TIPO_DE_CALLE': 'Tipo_de_Calle', 'Dirección Normalizada':'Dirección_Normalizada', 'COMUNA':'Comuna', 'pos x': 'Longitud', 'pos y': 'Latitud', 'PARTICIPANTES': 'Participantes', 'ACUSADO': 'Acusado', 'geo plana X': 'Longitud_Plana', 'geo plana Y': 'Latitud_Plana'  }
hechos_victimas = hechos_victimas.rename(columns=rename)

In [57]:
hechos_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     717 non-null    object        
 1   Fecha                  717 non-null    datetime64[ns]
 2   Año                    717 non-null    int64         
 3   Mes                    717 non-null    int64         
 4   Dia                    717 non-null    int64         
 5   Rol                    717 non-null    object        
 6   Victima                717 non-null    object        
 7   Sexo                   717 non-null    object        
 8   Edad                   717 non-null    int64         
 9   Fecha_Fallecimiento    717 non-null    datetime64[ns]
 10  Numero_Victimas        717 non-null    int64         
 11  Franja_Horaria         717 non-null    int32         
 12  Lugar_Del_Hecho        717 non-null    object        
 13  Tipo_

Eliminaremos la longitud_plana y latitud_plana pues no es la que usa PowerBi en sus tipos de mapas

In [58]:
hechos_victimas.drop(columns=['Longitud_Plana', 'Latitud_Plana'], inplace=True)

In [59]:
hechos_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     717 non-null    object        
 1   Fecha                  717 non-null    datetime64[ns]
 2   Año                    717 non-null    int64         
 3   Mes                    717 non-null    int64         
 4   Dia                    717 non-null    int64         
 5   Rol                    717 non-null    object        
 6   Victima                717 non-null    object        
 7   Sexo                   717 non-null    object        
 8   Edad                   717 non-null    int64         
 9   Fecha_Fallecimiento    717 non-null    datetime64[ns]
 10  Numero_Victimas        717 non-null    int64         
 11  Franja_Horaria         717 non-null    int32         
 12  Lugar_Del_Hecho        717 non-null    object        
 13  Tipo_

Vamos a revisar la columna acusado comparándola con participantes, rol y víctima

In [60]:
a = hechos_victimas['Acusado'].value_counts()
a

Acusado
AUTO           210
PASAJEROS      178
CARGAS         150
OBJETO FIJO     67
MOTO            58
SD              23
MULTIPLE        17
BICICLETA        7
OTRO             6
TREN             1
Name: count, dtype: int64

Vamos a explorar los SD, como podemos observar la dinámica de la columna participantes es que como dato inicial esta la víctima y como dato final el acusado

In [61]:
filtro = hechos_victimas[hechos_victimas['Acusado'] == 'SD']
filtro = filtro[['Acusado', 'Participantes', 'Rol', 'Victima']]
filtro

,Acusado,Participantes,Rol,Victima
3,SD,MOTO-SD,CONDUCTOR,MOTO
33,SD,MOTO-SD,CONDUCTOR,MOTO
36,SD,SD-SD,CONDUCTOR,MOTO
39,SD,MOTO-SD,CONDUCTOR,MOTO
58,SD,AUTO-SD,CONDUCTOR,AUTO
77,SD,SD-SD,CONDUCTOR,MOTO
84,SD,PEATON-SD,PEATON,PEATON
93,SD,SD-SD,PASAJERO_ACOMPAÑANTE,MOTO
107,SD,PEATON-SD,PEATON,PEATON
114,SD,MOTO-SD,CONDUCTOR,MOTO


Basados en lo anterior, y sabiendo que en la mayoría de los casos la culpabilidad la tiene el AUTO y analizando los tipos de víctimas para los casos donde no conocemos el acusado, podemos inferir que en gran parte esos accidentes de debieron a un auto. Procederemos a imputar en la columna acusado para los 25 casos faltantes como responsable el AUTO.

In [62]:
hechos_victimas['Acusado'] = hechos_victimas['Acusado'].replace('SD', 'AUTO')

In [63]:
filtro = hechos_victimas[hechos_victimas['Acusado'] == 'SD']
filtro = filtro[['Acusado', 'Participantes', 'Rol', 'Victima']]
filtro

,Acusado,Participantes,Rol,Victima


En las columnas latitud y longitud identificamos celdas que a pesar de no estar vacías únicamente contienen un punto, vamos a reemplazar el mismo por un espacio vacío. No las eliminaremos pues a pesar de no tener la ubicación exacta del accidente no quiero afectar el conteo ni distribución de otras variables

In [64]:
filtro = hechos_victimas[hechos_victimas['Latitud'] == '.']
filtro

,ID,Fecha,Año,Mes,Dia,Rol,Victima,Sexo,Edad,Fecha_Fallecimiento,Numero_Victimas,Franja_Horaria,Lugar_Del_Hecho,Tipo_de_Calle,Dirección_Normalizada,Comuna,Longitud,Latitud,Participantes,Acusado
39,2016-0052,2016-04-20,2016,4,20,CONDUCTOR,MOTO,MASCULINO,40,2016-04-20,1,20,AUTOPISTA LUGONES PK 10000,AUTOPISTA,NaN,13,.,.,MOTO-SD,AUTO
108,2016-0136,2016-10-25,2016,10,25,CONDUCTOR,MOTO,MASCULINO,40,2016-10-25,1,0,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,NaN,4,.,.,MOTO-CARGAS,CARGAS
121,2016-0151,2016-11-18,2016,11,18,PEATON,PEATON,MASCULINO,40,2016-11-18,1,20,SD,CALLE,NaN,0,.,.,PEATON-SD,AUTO
141,2016-0174,2016-12-27,2016,12,27,CONDUCTOR,MOTO,MASCULINO,40,2016-12-27,1,0,AUTOPISTA 25 DE MAYO,AUTOPISTA,AUTOPISTA 25 DE MAYO,0,.,.,SD-SD,AUTO
182,2017-0042,2017-04-10,2017,4,10,CONDUCTOR,MOTO,MASCULINO,40,2017-04-10,1,9,AV. LEOPOLDO LUGONES PKM 6900,GRAL PAZ,"LUGONES, LEOPOLDO AV.",14,.,.,MOTO-CARGAS,CARGAS
186,2017-0050,2017-04-28,2017,4,28,CONDUCTOR,MOTO,MASCULINO,46,2017-04-28,2,11,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,NaN,9,.,.,MOTO-CARGAS,CARGAS
187,2017-0050,2017-04-28,2017,4,28,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,16,2017-04-28,2,11,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,NaN,9,.,.,MOTO-CARGAS,CARGAS
188,2017-0051,2017-05-01,2017,5,1,CONDUCTOR,AUTO,MASCULINO,33,2017-05-01,1,3,AU DELLEPIANE 2400,AUTOPISTA,NaN,7,.,.,AUTO-AUTO,AUTO
266,2017-0140,2017-11-19,2017,11,19,CONDUCTOR,MOTO,MASCULINO,24,2017-11-19,1,23,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,.,.,MOTO-PASAJEROS,PASAJEROS
327,2018-0039,2018-04-21,2018,4,21,PEATON,PEATON,MASCULINO,37,2018-04-21,1,22,AUTOPISTA LUGONES KM 4.7,AUTOPISTA,NaN,14,.,.,PEATON-AUTO,AUTO


In [65]:
hechos_victimas['Latitud'] = hechos_victimas['Latitud'].replace('.', '')
hechos_victimas['Longitud'] = hechos_victimas['Longitud'].replace('.', '')
filtro = hechos_victimas[hechos_victimas['Latitud'] == '.']
filtro

,ID,Fecha,Año,Mes,Dia,Rol,Victima,Sexo,Edad,Fecha_Fallecimiento,Numero_Victimas,Franja_Horaria,Lugar_Del_Hecho,Tipo_de_Calle,Dirección_Normalizada,Comuna,Longitud,Latitud,Participantes,Acusado


Crearemos 3 columnas para correspondientes al año, mes y día de fallecimiento

In [66]:
hechos_victimas['Fecha_Fallecimiento'] = pd.to_datetime(hechos_victimas['Fecha_Fallecimiento'])
hechos_victimas['Año_Fallecimiento'] = hechos_victimas['Fecha_Fallecimiento'].dt.year
hechos_victimas['Mes_Fallecimiento'] = hechos_victimas['Fecha_Fallecimiento'].dt.month
hechos_victimas['Dia_Fallecimiento'] = hechos_victimas['Fecha_Fallecimiento'].dt.day

In [67]:
hechos_victimas.head(5)

,ID,Fecha,Año,Mes,Dia,Rol,Victima,Sexo,Edad,Fecha_Fallecimiento,...,Tipo_de_Calle,Dirección_Normalizada,Comuna,Longitud,Latitud,Participantes,Acusado,Año_Fallecimiento,Mes_Fallecimiento,Dia_Fallecimiento
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01,...,AVENIDA,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,-58.47533969,-34.68757022,MOTO-AUTO,AUTO,2016,1,1
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02,...,GRAL PAZ,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,-58.50877521,-34.66977709,AUTO-PASAJEROS,PASAJEROS,2016,1,2
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03,...,AVENIDA,ENTRE RIOS AV. 2034,1,-58.39040293,-34.63189362,MOTO-AUTO,AUTO,2016,1,3
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,2016-01-10,...,AVENIDA,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,-58.46503904,-34.68092974,MOTO-SD,AUTO,2016,1,10
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01,...,AVENIDA,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,-58.38718297,-34.62246630,MOTO-PASAJEROS,PASAJEROS,2016,2,1


Ahora haremos un mapeo de las variables categóricas con el fin de poder calcular correlaciones en el EDA

Rol

In [68]:
a = hechos_victimas['Rol'].value_counts()
print(a)

mapeo = {'CONDUCTOR': 1,
         'PEATON': 2,
         'PASAJERO_ACOMPAÑANTE': 3 ,
         'CICLISTA': 4 }

hechos_victimas['Rol_Map'] = hechos_victimas['Rol'].map(mapeo)

Rol
CONDUCTOR               341
PEATON                  267
PASAJERO_ACOMPAÑANTE     80
CICLISTA                 29
Name: count, dtype: int64


Victima

In [69]:
a = hechos_victimas['Victima'].value_counts()
print(a)

mapeo = {'MOTO': 1,
         'PEATON': 2,
         'AUTO': 3 ,
         'BICICLETA': 4,
          'CARGAS': 5,
           'PASAJEROS': 6,
             'MOVIL': 7}

hechos_victimas['Victima_Map'] = hechos_victimas['Victima'].map(mapeo)

Victima
MOTO         312
PEATON       267
AUTO          94
BICICLETA     29
CARGAS         7
PASAJEROS      5
MOVIL          3
Name: count, dtype: int64


Sexo

In [70]:
a = hechos_victimas['Sexo'].value_counts()
print(a)

mapeo = {'MASCULINO': 1,
         'FEMENINO': 2}

hechos_victimas['Sexo_Map'] = hechos_victimas['Sexo'].map(mapeo)

Sexo
MASCULINO    551
FEMENINO     166
Name: count, dtype: int64


Participantes

In [71]:
a = hechos_victimas['Acusado'].value_counts()
print(a)

mapeo = {'MOTO': 1,
         'PEATON': 2,
         'AUTO': 3 ,
         'BICICLETA': 4,
          'CARGAS': 5,
           'PASAJEROS': 6,
             'MOVIL': 7,
             'OBJETO FIJO': 8,
             'MULTIPLE': 9,
             'OTRO': 10,
             'TREN': 11}

hechos_victimas['Acusado_Map'] = hechos_victimas['Acusado'].map(mapeo)

Acusado
AUTO           233
PASAJEROS      178
CARGAS         150
OBJETO FIJO     67
MOTO            58
MULTIPLE        17
BICICLETA        7
OTRO             6
TREN             1
Name: count, dtype: int64


In [72]:
a = hechos_victimas
a.head(5)

,ID,Fecha,Año,Mes,Dia,Rol,Victima,Sexo,Edad,Fecha_Fallecimiento,...,Latitud,Participantes,Acusado,Año_Fallecimiento,Mes_Fallecimiento,Dia_Fallecimiento,Rol_Map,Victima_Map,Sexo_Map,Acusado_Map
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01,...,-34.68757022,MOTO-AUTO,AUTO,2016,1,1,1,1,1,3
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02,...,-34.66977709,AUTO-PASAJEROS,PASAJEROS,2016,1,2,1,3,1,6
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03,...,-34.63189362,MOTO-AUTO,AUTO,2016,1,3,1,1,1,3
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,2016-01-10,...,-34.68092974,MOTO-SD,AUTO,2016,1,10,1,1,1,3
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01,...,-34.62246630,MOTO-PASAJEROS,PASAJEROS,2016,2,1,1,1,1,6


El Dataframe listo para el EDA se exporta a un archivo CSV

In [73]:
hechos_victimas.to_csv('hechosyvictimas.csv', index=False)